In [79]:
####importing the neccesary packages
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras
import numpy as np
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt


In [80]:
data_path = os.listdir("H:/UK/Ann/train")


In [81]:
sample_img = plt.imread("H:/UK/Ann/train/yawn/10.jpg")
sample_img

array([[[ 65,  60,  54],
        [ 65,  60,  54],
        [ 65,  60,  54],
        ...,
        [254, 254, 254],
        [255, 255, 255],
        [255, 255, 255]],

       [[ 65,  60,  54],
        [ 65,  60,  54],
        [ 65,  60,  54],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[ 65,  60,  54],
        [ 65,  60,  54],
        [ 65,  60,  54],
        ...,
        [255, 255, 255],
        [254, 254, 254],
        [254, 254, 254]],

       ...,

       [[ 22,  25,  34],
        [ 22,  25,  34],
        [ 23,  26,  35],
        ...,
        [157, 165, 178],
        [156, 164, 177],
        [156, 164, 177]],

       [[ 22,  25,  34],
        [ 22,  25,  34],
        [ 23,  26,  35],
        ...,
        [155, 163, 176],
        [155, 163, 176],
        [155, 163, 176]],

       [[ 22,  25,  34],
        [ 22,  25,  34],
        [ 23,  26,  35],
        ...,
        [154, 162, 175],
        [154, 162, 175],
        [155, 163, 176]]

In [82]:
def face_yawn(direc="H:/UK/Ann/train", fac_path="H:/UK/Ann/haarcascade_frontalface_default.xml"):
    yaw_no = []
    IMG_SIZE = 145
    categories = ["yawn", "no_yawn"]
    for category in categories:
        link_path = os.path.join(direc, category)
        class_num1 = categories.index(category)
        print(class_num1)
        for image in os.listdir(link_path):
            image_array = cv2.imread(os.path.join(link_path, image), cv2.IMREAD_COLOR)
            face_cascade = cv2.CascadeClassifier(fac_path)
            faces = face_cascade.detectMultiScale(image_array, 1.3, 5)
            for (x, y, w, h) in faces:
                img = cv2.rectangle(image_array, (x, y), (x+w, y+h), (0, 255, 0), 2)
                roi_color = img[y:y+h, x:x+w]
                resized_array = cv2.resize(roi_color, (IMG_SIZE, IMG_SIZE))
                yaw_no.append([resized_array, class_num1])
    return yaw_no


yawn_no_yawn = face_yawn()

0
1


In [83]:
def get_data(dir_path="H:/UK/Ann/train", face_cas="H:/UK/Ann/haarcascade_frontalface_default.xml", eye_cas="H:/UK/Ann/haarcascade.xml"):
    data_path = ['Closed', 'Open']
    IMG_SIZE = 145
    data = []
    for label in data_path:
        path = os.path.join(dir_path, label)
        class_num = data_path.index(label)
        class_num +=2
        print(class_num)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append([resized_array, class_num])
            except Exception as e:
                print(e)
    return data

In [84]:
data_train = get_data()


2
3


In [85]:
def append_data():
    yaw_no = face_yawn()
    data = get_data()
    yaw_no.extend(data)
    return np.array(yaw_no)

In [86]:
new_data = append_data()
  

0
1
2
3


C:\Users\91915\AppData\Local\Temp\ipykernel_23300\4262209164.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(yaw_no)


In [87]:
new_data.dtype

dtype('O')

In [88]:
X = []
y = []
for feature, label in new_data:
    X.append(feature)
    y.append(label)

In [89]:
X = np.array(X)
X = X.reshape(-1, 145, 145, 3)

In [90]:
labeling = LabelBinarizer()
y = labeling.fit_transform(y)

In [91]:
y = np.array(y)


In [92]:
seed = 42
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, test_size=test_size)

In [93]:
train_gen = ImageDataGenerator(rescale=1/255, zoom_range=0.2, horizontal_flip=True, rotation_range=30)
test_gen = ImageDataGenerator(rescale=1/255)

train_gen = train_gen.flow(np.array(X_train), y_train, shuffle=False)
test_gen = test_gen.flow(np.array(X_test), y_test, shuffle=False)

In [94]:
model_def = Sequential()

model_def.add(Conv2D(256, (3, 3), activation="relu", input_shape=X_train.shape[1:]))
model_def.add(MaxPooling2D(2, 2))

model_def.add(Conv2D(128, (3, 3), activation="relu"))
model_def.add(MaxPooling2D(2, 2))

model_def.add(Conv2D(64, (3, 3), activation="relu"))
model_def.add(MaxPooling2D(2, 2))

model_def.add(Conv2D(32, (3, 3), activation="relu"))
model_def.add(MaxPooling2D(2, 2))

model_def.add(Flatten())
model_def.add(Dropout(0.5))

model_def.add(Dense(64, activation="relu"))
model_def.add(Dense(4, activation="softmax"))

model_def.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

model_def.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 143, 143, 256)     7168      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 71, 71, 256)      0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 69, 69, 128)       295040    
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 34, 34, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 32, 32, 64)        73792     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 16, 16, 64)      

In [95]:
History = model_def.fit(train_gen, epochs=25, validation_data=test_gen, shuffle=True, validation_steps=len(test_gen))

Epoch 1/10
42/42 [==============================] - 173s 4s/step - loss: 0.9721 - accuracy: 0.5861 - val_loss: 0.7642 - val_accuracy: 0.7539
Epoch 2/10
42/42 [==============================] - 172s 4s/step - loss: 0.4348 - accuracy: 0.8181 - val_loss: 0.4577 - val_accuracy: 0.7923
Epoch 3/10
42/42 [==============================] - 184s 4s/step - loss: 0.3382 - accuracy: 0.8510 - val_loss: 0.2784 - val_accuracy: 0.8831
Epoch 4/10
42/42 [==============================] - 180s 4s/step - loss: 0.3099 - accuracy: 0.8750 - val_loss: 0.2729 - val_accuracy: 0.8988
Epoch 5/10
42/42 [==============================] - 171s 4s/step - loss: 0.2655 - accuracy: 0.8975 - val_loss: 0.2861 - val_accuracy: 0.9005
Epoch 6/10
42/42 [==============================] - 161s 4s/step - loss: 0.2470 - accuracy: 0.8997 - val_loss: 0.2763 - val_accuracy: 0.8935
Epoch 7/10
42/42 [==============================] - 171s 4s/step - loss: 0.2145 - accuracy: 0.9154 - val_loss: 0.2460 - val_accuracy: 0.9127
Epoch 8/10
42

In [104]:
accuracy = History.History ['accuracy']
val_accuracy = History.History ['val_accuracy']
loss = History.History['loss']
val_loss = History.History['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, "g", label="trainning accuracy")
plt.plot(epochs, val_accuracy, "r", label="validation accuracy")
plt.legend()
plt.show()

plt.plot(epochs, loss, "g", label="trainning loss")
plt.plot(epochs, val_loss, "r", label="validation loss")
plt.legend()
plt.show()

AttributeError: 'History' object has no attribute 'final'

In [34]:
model_def.save("dorwsii.h5")


In [102]:
prediction=np.argmax(model_def.predict(X_test), axis=-1)

18/18 [==============================] - 10s 581ms/step


In [98]:
labels_new = ["yawn", "no_yawn", "Closed", "Open"]
IMG_SIZE = 145
def predict(filepath, face_cas="H:/UK/Ann/haarcascade_frontalface_default.xml"):
    img_array = cv2.imread(filepath, cv2.IMREAD_COLOR)
    img_array = img_array / 255
    resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return resized_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

model_pre = tf.keras.models.load_model("C:/Users/91915/dorwsii.h5")

In [99]:
from sklearn.metrics import classification_report
print(classification_report(np.argmax(y_test, axis=1), prediction, target_names=labels_new))

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [41]:
prediction = model_pre.predict([predict("H:/UK/Ann/testing/test5cl.jpg")])
np.argmax(prediction)

1/1 [==============================] - 0s 204ms/step


2